In [54]:
import os
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, Trainer, TrainingArguments, BitsAndBytesConfig, Trainer, pipeline
from peft import LoraConfig
from datasets import Dataset
from trl import SFTTrainer
from datasets import load_dataset
from langchain.prompts.prompt import PromptTemplate
from datasets import load_dataset
import gc

model_name = "NousResearch/Llama-2-7b-chat-hf"


In [55]:
#Force garbage collection
gc.collect()

torch.cuda.empty_cache()
def display_cuda_memory():    
    print("\n--------------------------------------------------\n")
    print("torch.cuda.memory_allocated: %fGB"%(torch.cuda.memory_allocated(0)/1024/1024/1024))
    print("torch.cuda.memory_reserved: %fGB"%(torch.cuda.memory_reserved(0)/1024/1024/1024))
    print("torch.cuda.max_memory_reserved: %fGB"%(torch.cuda.max_memory_reserved(0)/1024/1024/1024))
    print("\n--------------------------------------------------\n")

In [56]:
quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=getattr(torch, "float16"),
    bnb_4bit_use_double_quant=False,
)

In [57]:
# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [58]:
model = AutoModelForCausalLM.from_pretrained(model_name, 
  quantization_config=quant_config, 
  device_map={"":0}, 
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [59]:
dataset = load_dataset('flytech/python-codes-25k', split='train')


In [60]:

def formatting_prompts_func(example):
    output_texts = []
    instruction = example['instruction'][0]
    for i in range(len(example['instruction'])):
        text = f"### Question: {example['instruction'][i]}\n ### Answer: {example['output'][i]}"
        output_texts.append(text)
    return output_texts

In [61]:
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

model.config.use_cache = False
model.config.pretraining_tp = 1

In [62]:
peft_params = LoraConfig(
    r=8,
    lora_alpha=16,
    lora_dropout=0.1,
    bias="none",
    task_type="CAUSAL_LM",
)

In [63]:
training_params = TrainingArguments(
    output_dir="./results",
    num_train_epochs=2,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    logging_steps=5,
    learning_rate=2e-4,
    fp16=True
)

trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_params,
    dataset_text_field="text",
    max_seq_length=None,
    tokenizer=tokenizer,
    args=training_params,
    packing=False,
    formatting_func=formatting_prompts_func,
)

trainer.train()

You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
5,1.786100
10,1.749800
15,1.631500
20,1.387800
25,1.521200
30,1.513400
35,1.443800
40,1.355100
45,1.281900
50,1.205400


TrainOutput(global_step=24812, training_loss=0.6644307201655857, metrics={'train_runtime': 31631.6609, 'train_samples_per_second': 3.138, 'train_steps_per_second': 0.784, 'total_flos': 3.076988800614728e+17, 'train_loss': 0.6644307201655857, 'epoch': 2.0})

In [65]:
new_model = "./finetune_modesl/llama2"

trainer.model.save_pretrained(new_model)
trainer.tokenizer.save_pretrained(new_model)

('./finetune_modesl/llama2/tokenizer_config.json',
 './finetune_modesl/llama2/special_tokens_map.json',
 './finetune_modesl/llama2/tokenizer.json')

In [66]:


prompt = """<start_of_turn>how to convert json to dataframe.<end_of_turn>
<start_of_turn>model"""
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200)
result = pipe(f"<s>[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

/home/qvr905/.conda/envs/unsloth/lib/python3.10/site-packages/transformers/generation/utils.py:1270: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation )
  warnings.warn(


<s>[INST] <start_of_turn>how to convert json to dataframe.<end_of_turn>
<start_of_turn>model [/INST]  To convert a JSON string to a Pandas DataFrame, you can use the `json` module in Python and the `pd.read\_json` function in Pandas. Here's an example:

```python
import json
import pandas as pd

# Load the JSON string
with open('data.json') as f:
 data = json.load(f)

# Convert the JSON string to a Pandas DataFrame
df = pd.read\_json(data)

print(df)
```

In this example, we open a JSON file named `data.json` and load it into a Python dictionary using the `json.load()` function. We then use the `pd.read\_json()` function to convert the dictionary
